# **Final Presentation**



In this notebook team Power_Factor will describe its developed package: the different functionalities, the completed assignments that led to the developement of the package as well as some code demonstrations and collaboration discussion

In [5]:
from typing import List, Tuple

import networkx as nx
import json
import pprint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import csr_array
from IPython.display import display
from scipy.sparse.csgraph import connected_components
from power_grid_model.utils import json_deserialize, json_serialize
from power_grid_model.validation import ValidationException, assert_valid_batch_data, assert_valid_input_data
from power_grid_model import (
    CalculationMethod,
    CalculationType,
    ComponentType,
    DatasetType,
    PowerGridModel,
    initialize_array,
)

# **Assignment 1 -graph processing**

In assignment 1 the task is to build a graph processing class. For a given undirected graph as input, there are two functionalities to implement:
- Find downstream vertices - given a specific edge from the graph, find the downstream vertices of the edge, including the downstream vertex of the edge itself
- Find alternative edges - given a specific enabled edge from the graph, returns a list that indicates which currently disabled edges can be enabled so that the graph is again fully connected and acyclic
            

# **Check input validity**

Before the either of the two functionalities are used, the input data must be checked for validity. There are 7 criteria that are evaluated:
1. The vertex (node) and edge ids should be unique
2. The number of edges (or connections between two separate vertices) should have the same as the number of edge ids
3. The vertices connected by the specified edges should be valid vertex ids
4. The number of enabled/disabled edges should be the same as the number of edge ids
5. The id of the source vertex should be a valid vertex id
6. The graph should be fully connected
7. The graph should not contain cycles

Furthermore, Find alternative edges functionality it is important to check if the edge to be disabled has a valid id and whether it is already disabled.

In [11]:
class IDNotFoundError(Exception):
    "Raised when a source or edge id is not found/valid"


class InputLengthDoesNotMatchError(Exception):
    "Raised when number of enabled and disabled edges does not match number of total edges or number of vertex pairs does not match number of edges"


class IDNotUniqueError(Exception):
    "Raised when vertex or edge ids are not unique"


class GraphNotFullyConnectedError(Exception):
    "Raised when graph contains more than 1 component"


class GraphCycleError(Exception):
    "Raised when graph contains a cycle"


class EdgeAlreadyDisabledError(Exception):
    "Edge is already disabled"


def check_unique(vertex_ids, edge_ids): #checks vertex ids or edge ids are unique
    ver = list(set(vertex_ids)) #a set is used because sets contain only unique values
    edge = list(set(edge_ids))
    if len(ver) != len(vertex_ids) or len(edge) != len(edge_ids):
        raise IDNotUniqueError("Vertex or edge ids are not unique")


def check_length_pairs(edge_vertex_id_pairs, edge_ids): #checks if number of vertex pairs matches the number of edges
    if len(edge_vertex_id_pairs) != len(edge_ids):
        raise InputLengthDoesNotMatchError("Number of vertex pairs does not match number of edges")


def check_found_pairs(edge_vertex_id_pairs, vertex_ids): #checks if all vertex pairs contain valid vertex ids
    if all(all(elem in vertex_ids for elem in t) for t in edge_vertex_id_pairs) == False:
        raise IDNotFoundError("Vertex id not found in edge array")
    

def check_length_enabled(edge_enabled, edge_ids): #checks if the number of enabled/disabled edges is the same as the number of edge ids
    if len(edge_enabled) != len(edge_ids):
        raise InputLengthDoesNotMatchError("Number of enabled and disabled edges does not match number of total edges")


def check_found_source(source_vertex_id, vertex_ids): #checks if the source vertex id is valid
    if source_vertex_id not in vertex_ids:
        raise IDNotFoundError("Source vertex id not found")


def check_connect(vertex_ids, edge_enabled, edge_vertex_id_pairs): #checks if graph is fully connected
    size = len(vertex_ids)
    sparseMatrix = [[0 for i in range(size)] for j in range(size)]
    for i in range(size):
        for j in range(size):
            if ((vertex_ids[i], vertex_ids[j]) in edge_vertex_id_pairs) and sparseMatrix[i][j] == 0:
                if edge_enabled[edge_vertex_id_pairs.index((vertex_ids[i], vertex_ids[j]))]:
                    sparseMatrix[i][j] = vertex_ids[j]
                    sparseMatrix[j][i] = vertex_ids[i]
            elif ((vertex_ids[j], vertex_ids[i]) in edge_vertex_id_pairs) and sparseMatrix[i][j] == 0:
                if edge_enabled[edge_vertex_id_pairs.index((vertex_ids[j], vertex_ids[i]))]:
                    sparseMatrix[i][j] = vertex_ids[j]
                    sparseMatrix[j][i] = vertex_ids[i]
    graph = csr_array(sparseMatrix)
    components = connected_components(graph)
    if components[0] > 1:
        raise GraphNotFullyConnectedError("Graph contains more than 1 component")


def check_cycle(edge_enabled, edge_vertex_id_pairs): #checks if graph contains cycles
    G = nx.Graph()
    for (u, v), enabled in zip(edge_vertex_id_pairs, edge_enabled):
        if enabled:
            G.add_edge(u, v)

    has_cycle = nx.is_forest(G)  # A forest is a graph with no undirected cycles
    if has_cycle == False:
        raise GraphCycleError("Graph contains a cycle")


def check_found_edges(disabled_edge_id, all_edges): #checks if the edge to be disabled has a valid edge id
    if disabled_edge_id not in all_edges:
        raise IDNotFoundError("Disabled edge id not found in edge array")
    

def check_disabled(disabled_edge_id, edge_ids, edge_enabled): #checks if the edge to be disabled is already disabled
    if edge_enabled[edge_ids.index(disabled_edge_id)] == False:
        raise EdgeAlreadyDisabledError("Edge is already disabled")

With all of the necessary needed external functions declared, the graph processing class can now also be created

In [51]:
class GraphProcessor(nx.Graph):
    def __init__(
        self,
        vertex_ids: List[int],
        edge_ids: List[int],
        edge_vertex_id_pairs: List[Tuple[int, int]],
        edge_enabled: List[bool],
        source_vertex_id: int,
    ) -> None:
        super().__init__()
        check_unique(vertex_ids, edge_ids)
        check_length_pairs(edge_vertex_id_pairs, edge_ids)
        check_found_pairs(edge_vertex_id_pairs, vertex_ids)
        check_length_enabled(edge_enabled, edge_ids)
        check_found_source(source_vertex_id, vertex_ids)
        check_connect(vertex_ids, edge_enabled, edge_vertex_id_pairs)
        check_cycle(edge_enabled, edge_vertex_id_pairs)

        self.source_vertex_id = source_vertex_id
        self.vertex_ids = vertex_ids
        self.edge_enabled = edge_enabled
        self.edge_ids = edge_ids
        self.edge_vertex_id_pairs = edge_vertex_id_pairs
        self.add_nodes_from(vertex_ids)
        for i, (u, v) in enumerate(edge_vertex_id_pairs):
            self.add_edge(u, v, id=edge_ids[i], enabled=edge_enabled[i])

        self.enabled_subgraph = nx.Graph()
        for (u, v), enabled in zip(edge_vertex_id_pairs, edge_enabled):
            if enabled:
                self.enabled_subgraph.add_edge(u, v)
        # DFS tree & parent map from source
        self.dfs_tree = nx.dfs_tree(self.enabled_subgraph, self.source_vertex_id)
        self.parent_map = {child: parent for parent, child in nx.dfs_edges(self.dfs_tree, source=self.source_vertex_id)}

    def find_downstream_vertices(self, edge_id: int) -> List[int]:
        if edge_id not in self.edge_ids:
            raise IDNotFoundError("Edge ID not found.")

        edge_index = self.edge_ids.index(edge_id)
        if not self.edge_enabled[edge_index]:
            return []

        u, v = self.edge_vertex_id_pairs[edge_index]

        # Ensure both u and v are reachable
        if u not in self.dfs_tree or v not in self.dfs_tree:
            return []

        # Determine downstream vertex (child in DFS tree)
        if self.parent_map.get(v) == u:
            downstream_root = v
        elif self.parent_map.get(u) == v:
            downstream_root = u
        else:
            # If neither is parent of the other, one of them is ancestor; pick the child
            # or fallback to whichever is deeper
            depth = nx.single_source_shortest_path_length(self.dfs_tree, self.source_vertex_id)
            downstream_root = v if depth.get(v, 0) > depth.get(u, 0) else u

        descendants = list(nx.descendants(self.dfs_tree, downstream_root))
        return [downstream_root] + descendants

    def find_alternative_edges(self, disabled_edge_id: int) -> List[int]:
        ans = []
        check_found_edges(disabled_edge_id, self.edge_ids)
        check_disabled(disabled_edge_id, self.edge_ids, self.edge_enabled)
        H = nx.Graph()
        for i, (u, v) in enumerate(self.edge_vertex_id_pairs):
            if self.edge_enabled[i] == True and self.edge_ids[i] != disabled_edge_id:
                H.add_edge(u, v)
        for i, (u, v) in enumerate(self.edge_vertex_id_pairs):
            if self.edge_enabled[i] == False and self.edge_ids[i] != disabled_edge_id:
                H.add_edge(u, v)
                if nx.number_connected_components(H) == 1 and nx.is_forest(H):
                    ans.append(self.edge_ids[i])
                H.remove_edge(u, v)
        return ans

With the class implemented its functionalities can now be tested.

In [52]:
vertex_ids1 = [0, 2, 4, 6, 10]
edge_ids1 = [1, 3, 5, 7, 9, 8]
edge_vertex_id_pairs1 = [(0, 2), (0, 4), (0, 6), (2, 4), (2, 10), (4, 6)]
edge_enabled1 = [True, True, True, False, True, False]
source_vertex_id1 = 0

"""
vertex_0 (source) --edge_1(enabled)-- vertex_2 --edge_9(enabled)-- vertex_10
|                               |
|                           edge_7(disabled)
|                               |
-----------edge_3(enabled)-- vertex_4
|                               |
|                           edge_8(disabled)
|                               |
-----------edge_5(enabled)-- vertex_6
"""

test_graph1=GraphProcessor(vertex_ids1, edge_ids1, edge_vertex_id_pairs1, edge_enabled1, source_vertex_id1)
edge_to_disable=3
print(f"The list of alternative edges when disabling edge {edge_to_disable} is: {test_graph1.find_alternative_edges(edge_to_disable)}")


The list of alternative edges when disabling edge 3 is: [7, 8]


In [14]:
vertex_ids2 = [0, 2, 4, 6, 8, 10, 12]
edge_ids2 = [1, 3, 5, 7, 9, 11]
edge_vertex_id_pairs2 = [(0, 2), (2, 4), (2, 6), (4, 8), (8, 10), (6, 12)]
edge_enabled2 = [True, True, True, True, True, True]
source_vertex_id2 = 0

"""
    vertex_0 (source) --edge_1-- vertex_2 --edge_3-- vertex_4--edge 7--vertex 8 --edge 9--vertex 10
                                    |
                                  edge 5
                                    |
                                 vertex 6  --edge 11 --vertex 12
"""

test_graph2=GraphProcessor(vertex_ids2, edge_ids2, edge_vertex_id_pairs2, edge_enabled2, source_vertex_id2)
edge_down=3
print(f"The list of downstream vertices for edge {edge_down} is {test_graph2.find_downstream_vertices(edge_down)}")

The list of downstream vertices for edge 3 is [4, 8, 10]


# **Assignment 2 - power grid model**

# **Assignment 3 - developing a power system simulation package**

Now that graph processing and power grid model have been implemented, they can be combined to form a functional user package to simulate power systems. The package has 4 functionalities:

1. Input data validity check
2. EV penetration level
3. Optimal tap position
4. N-1 calculation

In [44]:
with open("C:/Users/Kossyo/Desktop/Уни/Power System computation and simulation/big_network/input/input_network_data.json") as fp:
    data = fp.read()

input_data = json_deserialize(data)

with open("C:/Users/Kossyo/Desktop/Уни/Power System computation and simulation/big_network/input/meta_data.json") as fp:
    meta = fp.read()

meta_data = json.loads(meta)
pprint.pprint(meta_data)

active_power_profile = pd.read_parquet("C:/Users/Kossyo/Desktop/Уни/Power System computation and simulation/big_network/input/active_power_profile.parquet")
reactive_power_profile = pd.read_parquet("C:/Users/Kossyo/Desktop/Уни/Power System computation and simulation/big_network/input/active_power_profile.parquet")
ev_active_power_profile = pd.read_parquet("C:/Users/Kossyo/Desktop/Уни/Power System computation and simulation/big_network/input/active_power_profile.parquet")

dtype = {
    "names": [
        "id",
        "from_node",
        "to_node",
        "from_status",
        "to_status",
        "r1",
        "x1",
        "c1",
        "tan1",
        "r0",
        "x0",
        "c0",
        "tan0",
        "i_n",
    ]
}
df = pd.DataFrame(
    input_data[ComponentType.line], columns=dtype["names"]
)  # get the data for the lines of the grid as a dataframe

{'lv_busbar': 1,
 'lv_feeders': [1204, 1304, 1404, 1504, 1604, 1704, 1804, 1904],
 'mv_source_node': 0,
 'source': 802,
 'transformer': 803}


# **Input data validity check**

Check the following validity criteria for the input data. Raise or passthrough relevant errors.

In [16]:
class MoreThanOneTransformerOrSource(Exception):
    "Raised when there is more than one transformer or source in meta_data.json"


class InvalidLVIds(Exception):
    "Raised when LV Feeder IDs are not valid line IDs."


class NonMatchingTransformerLineNodes(Exception):
    "Raised when the lines in the LV Feeder IDs do not have the from_node the same as the to_node of the transformer."


class NonMatchingTimestamps(Exception):
    "Raised when the timestamps are not matching between the active load profile, reactive load profile, and EV charging profile."


class InvalidProfileIds(Exception):
    "Raised when the IDs in active load profile and reactive load profile are not matching."


class InvalidSymloadIds(Exception):
    "Raised when the IDs in active load profile and reactive load profile are not matching the symload IDs."


class InvalidNumberOfEVProfiles(Exception):
    "raised when the number of EV charging profile is at least the same as the number of sym_load."


class InvalidLineIds(Exception):
    "Raised when the given Line ID to disconnect is not a valid"


class NonConnnected(Exception):
    "Raised when the given Line ID is not connected at both sides in the base case"


class InvalidCriteria(Exception):
    "Raised when the criteria for optimal tap position is not valid. Use 'Voltage_deviation' or 'Total_loss'."


def check_source_transformer(meta_data):
    if (type(meta_data["source"]) is not int) or (type(meta_data["transformer"]) is not int):
        raise MoreThanOneTransformerOrSource("LV grid contains more than one source or transformer")


def check_valid_LV_ids(LV_ids, Line_ids):
    if not all(item in Line_ids for item in LV_ids):
        raise InvalidLVIds("LV feeders contain invalid ids")


def check_line_transformer_nodes(lines_from_nodes, transformer_to_node):
    if not all(element == transformer_to_node for element in lines_from_nodes):
        raise NonMatchingTransformerLineNodes(
            "The lines in the LV Feeder IDs do not have the from_node the same as the to_node of the transformer"
        )


def check_timestamps(active_timestamp, reactive_timestamp, ev_timestamp):
    if not (active_timestamp.equals(reactive_timestamp) and active_timestamp.equals(ev_timestamp)):
        raise NonMatchingTimestamps("Timestamps between the active, reactive and ev profiles do not match")


def check_profile_ids(active_ids, reactive_ids):
    if not active_ids.equals(reactive_ids):
        raise InvalidProfileIds("The active and reactive load profile IDs are not matching")


def check_symload_ids(active_ids, reactive_ids, symload_ids):
    if not (all(item in symload_ids for item in active_ids) and (all(item in symload_ids for item in reactive_ids))):
        raise InvalidSymloadIds("The active and reactive load profile IDs are not matching the sym_load IDs")


def check_number_of_ev_profiles(ev_profiles, symload_profiles):
    if len(ev_profiles) > len(symload_profiles):
        raise InvalidNumberOfEVProfiles("The number of EV charging profile is larger than the number of sym_loads")


def check_valid_line_ids(id, Line_ids):
    if id not in Line_ids:
        raise InvalidLineIds("Invalid Line ID")


def check_line_id_connected(fr, to):
    if not (fr.item() == 1 and to.item() == 1):
        raise NonConnnected("Line ID not connected at both sides in the base case")

In [17]:
def input_data_validity_check(input_data, meta_data):

    assert_valid_input_data(
        input_data=input_data, calculation_type=CalculationType.power_flow
    )  # check if input data is valid
    check_source_transformer(meta_data)  # check if LV grid has exactly one transformer, and one source.

    check_valid_LV_ids(
        meta_data["lv_feeders"], input_data[ComponentType.line]["id"]
    )  # check if All IDs in the LV Feeder IDs are valid line IDs
    check_line_transformer_nodes(
        df[df["id"].isin(meta_data["lv_feeders"])]["from_node"].tolist(),
        input_data[ComponentType.transformer]["to_node"],
    )  # check if all the lines in the LV Feeder IDs have the from_node the same as the to_node of the transformer

    transformer_tuple = list(
        zip(
            input_data[ComponentType.transformer]["from_node"].tolist(),
            input_data[ComponentType.transformer]["to_node"].tolist(),
        )
    )  # transformer also connects two nodes
    line_nodes_id_pairs = (
        list(
            zip(
                input_data[ComponentType.line]["from_node"].tolist(), input_data[ComponentType.line]["to_node"].tolist()
            )
        )
        + transformer_tuple
    )  # add nodes connected by transformer to list of lines connecting nodes
    status_list = list(input_data[ComponentType.line]["to_status"].tolist()) + list(
        input_data[ComponentType.transformer]["to_status"].tolist()
    )  # add transformer connection status to list of lines' statuses

    check_connect(
        input_data[ComponentType.node]["id"], status_list, line_nodes_id_pairs
    )  # check if the grid is fully connected in the initial state
    check_cycle(status_list, line_nodes_id_pairs)  # check if the grid has no cycles in the initial state

    check_timestamps(
        active_power_profile.index, reactive_power_profile.index, ev_active_power_profile.index
    )  # checks if timestamps are matching
    check_profile_ids(
        active_power_profile.columns, reactive_power_profile.columns
    )  # checks if number of active and reactive profiles are matching
    check_symload_ids(
        active_power_profile.columns, reactive_power_profile.columns, input_data[ComponentType.sym_load]["id"]
    )  # checks if IDs are matching
    check_number_of_ev_profiles(
        ev_active_power_profile.columns, input_data[ComponentType.sym_load]["id"]
    )  # checks if number of EV profiles does not exceed number of sym_loads
    print("Input data is valid!")


Validity check tested on big network

In [18]:
input_data_validity_check(input_data, meta_data)

Input data is valid!


# **EV penetration**

Given a (user-provided) input of electrical vehicle (EV) penetration level, i.e. the percentage of houses which has EV charged at home, randomly add EV charging profiles to the houses according to the following creteria.

# **Optimal tap position**

In this functionality, the user would like to optimize the tap position of the transformer in the LV grid.

# **N-1 calculation**

In this functionality, the user would like to know alternative grid topology when a given line is out of service. The user will provide the Line ID which is going to be out of service.

In [54]:
def check_valid_line_ids(id, Line_ids):
    if id not in Line_ids:
        raise InvalidLineIds("Invalid Line ID")


def check_line_id_connected(fr, to):
    if not (fr.item() == 1 and to.item() == 1):
        raise NonConnnected("Line ID not connected at both sides in the base case")


def find_alternative_lines(
    vertex_ids, edge_ids, edge_vertex_id_pairs, edge_enabled, source_vertex_id, id_to_disconnect
):
    test = GraphProcessor(
        vertex_ids, edge_ids, edge_vertex_id_pairs, edge_enabled, source_vertex_id
    )  # create a graph from the provided data about nodes and lines
    return test.find_alternative_edges(
        id_to_disconnect
    )  # find the alternative edges to make the graph fully connected, when the line is disconnected


def power_flow_calc(active_power_profile, alt_lines_list, line_id_list):

    load_profile_active = initialize_array(DatasetType.update, ComponentType.sym_load, active_power_profile.shape)
    load_profile_active["id"] = active_power_profile.columns.to_numpy()
    load_profile_active["p_specified"] = active_power_profile.to_numpy()
    load_profile_active["q_specified"] = 0.0
    update_data = {ComponentType.sym_load: load_profile_active}

    input_data[ComponentType.line]["from_status"][
        line_id_list.index(id_to_disconnect)
    ] = 0  # disconnect the line that the user wants to disconnect
    input_data[ComponentType.line]["to_status"][
        line_id_list.index(id_to_disconnect)
    ] = 0  # disconnect the line that the user wants to disconnect

    max_loading_alt = np.zeros(len(alt_lines_list))  # initialize max_loading column values
    max_line_alt = np.zeros(len(alt_lines_list))  # initialize max_line_alt column values
    max_loading_timestamp_alt = np.zeros(
        len(alt_lines_list), dtype=object
    )  # initialize max_loading_timestamp column values
    for k in range(len(alt_lines_list)):
        input_data[ComponentType.line]["from_status"][k] = 1
        input_data[ComponentType.line]["to_status"][k] = 1  # connect the kth alternative line
        model = PowerGridModel(input_data=input_data)
        result = model.calculate_power_flow(
            update_data=update_data, calculation_method=CalculationMethod.newton_raphson
        )  # perform the power flow analysis when the kth line is connected
        # print(alt_lines_list[k])
        # print(result)
        ids = np.unique(result[ComponentType.line]["id"])
        max_loading = np.zeros(len(ids))
        max_loading_timestamp = np.zeros(len(ids), dtype=object)
        temp_max = -99999999
        for i in range(len(ids)):
            max_loading[i] = result[ComponentType.line]["loading"][:, i].max()
            for j in range(len(active_power_profile.index)):
                if max_loading[i] == result[ComponentType.line]["loading"][j, i]:
                    max_loading_timestamp[i] = active_power_profile.index[j]
                    if max_loading[i] > temp_max:
                        temp_max = max_loading[i]
                        max_loading_alt[k] = max_loading[i]
                        max_line_alt[k] = result[ComponentType.line]["id"][j, i]
                        max_loading_timestamp_alt[k] = active_power_profile.index[j]
                        # print(active_power_profile.index[j])
        input_data[ComponentType.line]["from_status"][k] = 0
        input_data[ComponentType.line]["to_status"][k] = 0  # disconnect kth line

    input_data[ComponentType.line]["from_status"][
        line_id_list.index(id_to_disconnect)
    ] = 1  # reconnect the line that the user wants to disconnect
    input_data[ComponentType.line]["to_status"][
        line_id_list.index(id_to_disconnect)
    ] = 1  # reconnect the line that the user wants to disconnect

    output_data = pd.DataFrame()  # generate specified table from assignment 3
    output_data["Alternative line ID"] = alt_lines_list
    output_data["Max_loading"] = max_loading_alt
    output_data["Max_line_id"] = max_line_alt
    output_data["Max_timestamp"] = max_loading_timestamp_alt
    display(output_data)
    return output_data


In [55]:
def N_minus_one_calculation(id_to_disconnect):

    check_valid_line_ids(id_to_disconnect, input_data[ComponentType.line]["id"])  # check if ID is valid
    check_line_id_connected(
        df[df["id"] == id_to_disconnect]["from_status"], df[df["id"] == id_to_disconnect]["to_status"]
    )  # check if line with selected ID is connected

    transformer_tuple = list(
        zip(
            input_data[ComponentType.transformer]["from_node"].tolist(),
            input_data[ComponentType.transformer]["to_node"].tolist(),
        )
    )  # transformer also connects two nodes
    line_nodes_id_pairs = (
        list(
            zip(
                input_data[ComponentType.line]["from_node"].tolist(), input_data[ComponentType.line]["to_node"].tolist()
            )
        )
        + transformer_tuple
    )  # add nodes connected by transformer to list of lines connecting nodes
    status_list = list(df["to_status"].tolist()) + list(
        input_data[ComponentType.transformer]["to_status"].tolist()
    )  # add transformer connection status to list of lines' statuses
    line_id_list = df["id"].tolist()
    new_id = (df["id"].iloc[-1] + 1).tolist()
    line_id_list.append(new_id)  # add another line id to mimic the transformer connection
    alt_lines_list = find_alternative_lines(
        input_data[ComponentType.node]["id"].tolist(),
        line_id_list,
        line_nodes_id_pairs,
        status_list,
        meta_data["mv_source_node"],
        id_to_disconnect,
    )

    print(
        f"To make the grid fully connected, the following lines need to be connected: {alt_lines_list}"
    )  # find alternative currently disconnected lines to make the grid fully connected

    power_flow_calc(active_power_profile, alt_lines_list, line_id_list)

In [58]:
id_to_disconnect = 2002 #returns [2010] list and a table of 1 row
N_minus_one_calculation(id_to_disconnect) 

To make the grid fully connected, the following lines need to be connected: [2010]


,Alternative line ID,Max_loading,Max_line_id,Max_timestamp
0,2010,0.073859,1906.0,2025-11-05 06:45:00
